# Gaps among primes - closing the cycle G(p#)
For the cycle $\mathcal{G}(p^\#)$, let $q=g_1+1$ be the next prime.  All of the gaps from $q$ through $q^2$
will survive the sieve as gaps between primes.  We may not be able to create the subsequent cycles of gaps, but
we can enforce the fusions within $\mathcal{G}(p^\#)$ to confirm the gaps among primes, from $q$ through $p^\#$.

The code in this notebook will close the cycles of gaps up through $\mathcal{G}(23^\#)$.  NOTE that the outputs
are available in the npy-files.


In [1]:
import pandas as pd
import numpy as np
import array
from plotnine import ggplot, geom_point, aes, geom_line, theme, ggsave
import gc
import psutil
import sys

In [2]:
G23= np.load('G23uint.npy')

## NOTE: This approach works up through $\mathcal{G}(23^\#)$.
From 29 on, the processing time for this approach does not scale.  The large arrays seem to be drastically slowing down the kernel.
We have written a special notebook for closing $\mathcal{G}(29^\#)$ -- not this one!

In [ ]:
# block to check the available system memory
gc.collect()
memory = psutil.virtual_memory()
available_memory = memory.available
del memory
print(f"Available memory: {available_memory / (1024 ** 2):.2f} MB")

In [4]:
# function for closing a cycle
# we assume that Gin is (the front of) a cycle of gaps
# NOTE: this version uses modular sums in order to avoid scalar overflows.
def closeG(Gin):
    lenG = len(Gin)
    try:
        Gout = Gin.copy()  # start with a complete copy of the input array Gin
    except:
        print("Memory error in closeG")
        return
    # to avoid overflow in sum() we convert the elements of Gout to integer
    spanG = np.sum(Gout, dtype=int)  
    maxp = int(np.sqrt(spanG)) # bound on the Horizon of survival
    # initialize variables for the first pass through the main loop
    q = int(Gin[0]+1)  # initiate the prime q
    iq = int(0) # index of the prime we're on
    jg = 1
    headG = 1  # start beyond the current prime - we don't want to aggregate the first gap
    tailG = lenG-1 # end of the effective cycle Gout
    '''
    # A different approach to the fusions in step R3, we want to preserve the gaps at the front of the cycle.
    # instead of aggregating them into gap g1.
    # And without the concatentations of step R2, the fusions stop at the end of the array Gin.
    # Mindful of possible memory constraints, we perform the fusions in-place within Gout
    # We use residues mod q to find multiples
    '''
    while (q <= maxp):
        # loop through multiples of q, using jq, identify fusions
        firstflag = True
        jg = headG + 1
        residuesum = (1+np.sum(Gout[0:(jg+1)], dtype=int)) % q  
        print(f"Closing over prime {q} of {maxp}, head {iq}, tail {tailG}, jg {jg}, residue {residuesum}", end='\r')
        # first fusion will be at q^2.  We preserve the gaps at front of cycle.

        while (jg < tailG):
            jg += 1
            residuesum = (residuesum + Gout[jg])% q
            if (residuesum == 0 and jg < tailG): # multiple of q
                Gout[jg] = Gout[jg] + Gout[jg+1]
                residuesum = (Gout[jg+1])% q
                Gout[jg+1] = 0
                jg += 1
                if (firstflag):
                    firstflag = False
                    headG = jg

        # End of loop for multiples of q
        Gout = [x for x in Gout if x > 0]
        # Prepare for next pass through q-loop
        iq += 1
        q += Gout[iq]
        tailG = len(Gout)-1
        if ((iq % 1000) == 0):
            gc.collect()
    # end of q-loop -- Gin should be closed at this point

    return(Gout)
    

In [5]:
G23[0:100]

array([30,  6,  4,  2,  4,  6,  6,  2,  6,  4,  2,  6,  4,  6,  8,  4,  2,
        4,  2,  4, 14,  4,  6,  2, 10,  2,  6,  6,  4,  6,  6,  2, 10,  2,
        4,  2, 12, 12,  4,  2,  4,  6,  2, 10,  6,  6,  6,  2,  6,  4,  2,
       10, 14,  4,  2,  4, 14,  6, 10,  2,  4,  6,  8,  6,  6,  4,  6,  8,
        4,  8, 10,  2, 10,  2,  6,  4,  6,  8,  4,  2,  4, 12,  8,  4,  8,
        4,  6, 12,  2, 18,  6, 10,  6,  6,  2,  6, 10,  6,  6,  2],
      dtype=uint16)

In [ ]:
cG23 = closeG(G23)

Closing over prime 1973 of 80434, head 287, tail 481598042, jg 270407, residue 5306

In [7]:
primes23 = np.zeros((len(cG23)-1), dtype=int) # can't trust the last gap
primes23[0] = 1+cG23[0]
i = 1
while (i < len(primes23)):
    primes23[i] = primes23[i-1]+cG23[i]
    i += 1

In [8]:
print(f"Cycle length {len(G23)} survivors length {len(cG23)}, primes {len(primes23)}")

Cycle length 36495360 survivors length 12283523, primes 12283522


In [9]:
primes29[0:200]

array([  29,   31,   37,   41,   43,   47,   53,   59,   61,   67,   71,
         73,   79,   83,   89,   97,  101,  103,  107,  109,  113,  127,
        131,  137,  139,  149,  151,  157,  163,  167,  173,  179,  181,
        191,  193,  197,  199,  211,  223,  227,  229,  233,  239,  241,
        251,  257,  263,  269,  271,  277,  281,  283,  293,  307,  311,
        313,  317,  331,  337,  347,  349,  353,  359,  367,  373,  379,
        383,  389,  397,  401,  409,  419,  421,  431,  433,  439,  443,
        449,  457,  461,  463,  467,  479,  487,  491,  499,  503,  509,
        521,  523,  541,  547,  557,  563,  569,  571,  577,  587,  593,
        599,  601,  607,  613,  617,  619,  631,  641,  643,  647,  653,
        659,  661,  673,  677,  683,  691,  701,  709,  719,  727,  733,
        739,  743,  751,  757,  761,  769,  773,  787,  797,  809,  811,
        821,  823,  827,  829,  839,  853,  857,  859,  863,  877,  881,
        883,  887,  907,  911,  919,  929,  937,  9

In [10]:
# primes23[44940:45140]

array([545023, 545029, 545033, 545057, 545063, 545087, 545089, 545093,
       545117, 545131, 545141, 545143, 545161, 545189, 545203, 545213,
       545231, 545239, 545257, 545267, 545291, 545329, 545371, 545387,
       545429, 545437, 545443, 545449, 545473, 545477, 545483, 545497,
       545521, 545527, 545533, 545543, 545549, 545551, 545579, 545599,
       545609, 545617, 545621, 545641, 545647, 545651, 545663, 545711,
       545723, 545731, 545747, 545749, 545759, 545773, 545789, 545791,
       545827, 545843, 545863, 545873, 545893, 545899, 545911, 545917,
       545929, 545933, 545939, 545947, 545959, 546001, 546017, 546019,
       546031, 546047, 546053, 546067, 546071, 546097, 546101, 546103,
       546109, 546137, 546149, 546151, 546173, 546179, 546197, 546211,
       546233, 546239, 546241, 546253, 546263, 546283, 546289, 546317,
       546323, 546341, 546349, 546353, 546361, 546367, 546373, 546391,
       546461, 546467, 546479, 546509, 546523, 546547, 546569, 546583,
      

In [ ]:
# primes23[12283450:12283521]

In [85]:
#primes23[12283450:12283521]

array([223091299, 223091303, 223091327, 223091333, 223091347, 223091359,
       223091381, 223091387, 223091437, 223091501, 223091503, 223091543,
       223091549, 223091551, 223091581, 223091587, 223091599, 223091621,
       223091623, 223091639, 223091747, 223091753, 223091797, 223091837,
       223091861, 223091873, 223091887, 223091917, 223091959, 223091963,
       223091989, 223092007, 223092017, 223092043, 223092047, 223092049,
       223092061, 223092073, 223092101, 223092109, 223092119, 223092169,
       223092209, 223092217, 223092227, 223092239, 223092257, 223092263,
       223092269, 223092301, 223092361, 223092431, 223092469, 223092481,
       223092497, 223092517, 223092559, 223092577, 223092587, 223092601,
       223092631, 223092671, 223092673, 223092689, 223092733, 223092743,
       223092757, 223092769, 223092773, 223092791, 223092809])

In [11]:
gc.collect()

0

In [87]:
# np.save('filename', array)